In [99]:
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
SERPER_API_KEY=os.getenv("SERPER_API_KEY")
os.environ["SERPER_API_KEY"] = SERPER_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [100]:
from langchain_google_genai import ChatGoogleGenerativeAI # Chat Model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

In [101]:
llm.invoke("hi").content

'Hi there! How can I help you today?\n'

In [102]:
message=[{"role":"system","content":"you are a helpful assistant"},
         {"role":"user","content":"hi how are you?"}]

In [103]:
llm.invoke(message)

AIMessage(content="Hi there! I'm doing well, thanks for asking. As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you with whatever you need. How are you doing today?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-5e2d83be-9acf-4e7f-85d6-113716e35d56-0', usage_metadata={'input_tokens': 11, 'output_tokens': 55, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}})

In [104]:
result = llm.invoke(message)
print(result.content)

Hi there! I'm doing well, thanks for asking. As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to help. How are you doing today?



In [105]:
class Chatbot:
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")
        result = llm.invoke(self.message)
        return result.content
        

In [106]:
bot=Chatbot(system="you are helpful assistant")

In [107]:
bot("hi how are you?")

"Hi there! I'm doing well, thank you for asking. As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?\n"

In [108]:
bot("Who are you?")

"I am a large language model, trained by Google. I'm designed to be a helpful and informative AI assistant. I can communicate and generate human-like text in response to a wide range of prompts and questions. \n\nThink of me as a tool that can help you with:\n\n* **Answering questions:** I can draw on a vast amount of information to provide answers on various topics.\n* **Generating text:** I can create different kinds of creative text formats, like poems, code, scripts, musical pieces, email, letters, etc.\n* **Translating languages:** I can translate between many different languages.\n* **Summarizing text:** I can condense long pieces of text into shorter summaries.\n* **And much more!**\n\nEssentially, I'm a complex computer program that has been trained on a massive dataset of text and code, allowing me to understand and generate human language.\n\nIs there anything else you'd like to know about me?\n"

In [109]:
# History
bot.message

[{'role': 'system', 'content': 'you are helpful assistant'},
 {'role': 'user', 'content': 'hi how are you?'},
 {'role': 'assistant',
  'content': "Hi there! I'm doing well, thank you for asking. As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you. How are you doing today? Is there anything I can help you with?\n"},
 {'role': 'user', 'content': 'Who are you?'},
 {'role': 'assistant',
  'content': "I am a large language model, trained by Google. I'm designed to be a helpful and informative AI assistant. I can communicate and generate human-like text in response to a wide range of prompts and questions. \n\nThink of me as a tool that can help you with:\n\n* **Answering questions:** I can draw on a vast amount of information to provide answers on various topics.\n* **Generating text:** I can create different kinds of creative text formats, like poems, code, scripts, musical pieces, email, letters, etc.\n* *

In [110]:
# React agent depend on the prompt
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

Pattern Breakdown:

- ^: This matches the start of a string. It means the string must begin with what follows.
- Action:: This is a literal match. It means the string must have the text "Action:" at the beginning.
- (\w+):
    -  The parentheses () define a capture group. This allows you to extract part of the string that matches this section.
    - \w+ matches one or more word characters (letters, digits, and underscores). This will capture a word that follows "Action:".
- :: This is a literal colon that separates the word matched by (\w+) from the rest of the string.
- (.*):
-- This is another capture group, where .* matches any character (.) zero or more times (*), which means it captures everything that comes after the second colon.

What does it do?
This regex is looking for a string that:

Starts with the text "Action:".
Has a word right after it, separated by a colon.
Then, after another colon, it captures everything that follow



Example:

Suppose we have the following string:
"Action:Delete:This is a delete action."

If we apply the regex pattern:
- "^Action:(\w+):(.*)"

Here's what happens:
- "^Action:" matches the start of the string.
- "(\w+)" captures the word "Delete".
- "::" matches the second colon.
- "(.*)" captures the rest of the string, "This is a delete action.".

So, the regex pattern extracts two parts from the string:
- "Delete" (the word after "Action:")
- "This is a delete action." (everything after the second colon)

In [111]:
import re
action_re = re.compile('^Action: (\w+): (.*)') # Identify the action

In [112]:
# Sample strings
text1 = "Action: Move: North"
text2 = "Action: Jump: High"
text3 = "Error: Not an Action"

In [113]:
# Testing the pattern
match1 = action_re.match(text1)

In [114]:
match1

<re.Match object; span=(0, 19), match='Action: Move: North'>

In [115]:
match1.group(1)

'Move'

In [116]:
match1.group(2)

'North'

In [117]:
match3 = action_re.match(text3) # Empty

In [ ]:
match3.group(1)
# Show error empty

In [119]:
# tools
#1  Wikipedia Api
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [120]:
#2 Blog Search from simon database
# Run SQL Query to run on database to get data
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [121]:
#3 Calculator
def calculate(number):
    return eval(number)


In [122]:
calculate("2+2")

4

In [123]:
# Mappig of all the tool
known_actions={
    "wikipedia":wikipedia,
    "Calculate":calculate,
    "simon_blog_search":simon_blog_search
}


In [138]:
# With the help of the loop i will acheive the loop

def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result
        
        

In [125]:
prompt

'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop your output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\n\nYour available actions are:\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point\nsyntax if necessary\n\nwikipedia:\ne.g. wikipedia: Django\nReturns a summary from searching Wikipedia\n\nsimon_blog_search:\ne.g. simon_blog_search: Python\nSearch Simon\'s blog for that term\n\nExample session:\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\nObservation: France is a country. The capital is Paris.\n\nYou then output:\nAnswer: The capital of France is Paris\n\nPlease Note: if you get 

In [126]:
bot =Chatbot(prompt)

In [127]:
# Pass query
bot("How are you")

'Thought: This is a basic conversation question, so I should respond accordingly.\nAction: None\nPAUSE\nObservation: None\nAnswer: I am good.\n'

In [128]:
bot("What is your job ")

'Thought: I need to explain what my purpose is.\nAction: None\nPAUSE\nObservation: None\nAnswer: I am a large language model, designed to help users with a variety of tasks, including answering questions, providing information, and completing creative writing tasks.\n'

In [129]:
# Example
# Break the query class
next_prompt="Tell me  current GDP of India from wikipedia"

In [130]:
bot(next_prompt)

'Thought: I need to use Wikipedia to find the current GDP of India.\nAction: wikipedia: India GDP\nPAUSE\n'

In [131]:
# Take the result to actions under query function
result=bot(next_prompt)
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

In [133]:
actions

[<re.Match object; span=(0, 28), match='Action: wikipedia: India GDP'>]

In [134]:
action, action_input = actions[0].groups() # Extract 'wikipedia: India GDP'

In [135]:
action # Tool

'wikipedia'

In [136]:
action_input

'India GDP'

In [139]:
query("hi")

Thought: This is a basic greeting. I should respond in kind.
Action: None
PAUSE
Observation: None
Answer: hi



'Thought: This is a basic greeting. I should respond in kind.\nAction: None\nPAUSE\nObservation: None\nAnswer: hi\n'

In [140]:
query("How are you ")

Thought: This is a basic conversation question, so I should respond appropriately.
Action: None
PAUSE
Observation: None
Answer: I am good.



'Thought: This is a basic conversation question, so I should respond appropriately.\nAction: None\nPAUSE\nObservation: None\nAnswer: I am good.\n'

In [143]:
query("Has Simon written about AI?")

Thought: I should search Simon's blog for mentions of AI.
Action: simon_blog_search: AI
PAUSE

 -- running simon_blog_search AI
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplete" is actually a pretty good analogy for how a lot of these thi

'Thought: Okay, it seems Simon has written at least two blog posts specifically addressing the term "Artificial Intelligence". He acknowledges that his initial post was flawed and has provided a follow-up post with a revised viewpoint. It\'s clear he\'s engaged with the topic and the different perspectives surrounding the term.\nAnswer: Yes, Simon has written about AI, including a post titled "It\'s OK to call it Artificial Intelligence" and a follow-up post titled "What I should have said about the term Artificial Intelligence".\n'

In [144]:
query("can you tell me capital of usa?")

Thought: I should look up the USA on Wikipedia to find its capital.
Action: wikipedia: USA
PAUSE

 -- running wikipedia USA
Observation: The United States of America (<span class="searchmatch">USA</span>), commonly known as the United States (U.S.) or America, is a country primarily located in North America. It is a federal
Thought: The Wikipedia entry was cut off, but I'm confident it will mention the capital. I'll try searching for "capital of USA" on Wikipedia.
Action: wikipedia: capital of USA
PAUSE

 -- running wikipedia capital of USA
Observation: Popularly known as <span class="searchmatch">USA</span> <span class="searchmatch">Capital</span>, hard money lender <span class="searchmatch">USA</span> Commercial Mortgage Co. (USACM) was a Las Vegas, Nevada based mortgage broker owned by Tom Hantges
Thought: That Wikipedia search didn't give me the capital. I need to be more specific and try to find the capital using a different query on wikipedia.
Action: wikipedia: Washington D.C.
P

'Thought: Okay, the Wikipedia entry for Washington D.C. clearly states it is the capital of the United States.\nAnswer: The capital of the USA is Washington D.C.\n'